*This notebook is originally from https://github.com/neubig/anlp-code* by Graham Neubig

# Build a Rule-based Sentiment Classifier

This is a notebook in which you can attempt to build a rule-based sentiment classifier. It will take in a text `X` and return a `label` of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

The only thing that you should change in this notebook is the following cell which contains two important elements. The first is `extract_features(X)`, which will extract a dictionary of (named) feature values from the text. You should create this by hand, and a simple example is shown for you. The second is `feature_weights`, a dictionary which will assign a weight to each extracted feature.

The final way the classifier decides whether to assign a positive, negative, or neutral label is by calculating the dot product `feature_weights * extract_features(X)`, and if the value is greater than zero, return 1, less than zero return -1, and if exactly zero return 0.

Let's have some fun trying to design a classifier 😁


In [12]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    
    # Count the number of "good words" and "bad words" in the text
    good_words = ['love', 'good', 'nice', 'great', 'enjoy', 'enjoyed']
    bad_words = ['hate', 'bad', 'terrible', 'disappointing', 'sad', 'lost', 'angry', "dull"]
    for x_word in x_split:
        if x_word in good_words:
            features['good_word_count'] = features.get('good_word_count', 0) + 1
        if x_word in bad_words:
            features['bad_word_count'] = features.get('bad_word_count', 0) + 1
    
    # The "bias" value is always one, to allow us to assign a "default" score to the text
    features['bias'] = 1
    
    return features

feature_weights = {'good_word_count': 1.0, 'bad_word_count': -1.0, 'bias': 0.5}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [4]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [3]:
x_train, y_train = read_xy_data('./data/train.txt')
x_test, y_test = read_xy_data('./data/dev.txt')

In [5]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Run the Classifier and Calculate Accuracy

Run the classifier over the training and dev (test) sets and calculate accuracy

In [14]:
def run_classifier(x: str) -> int:
    score = 0
    for feat_name, feat_value in extract_features(x).items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [15]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(x)
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [16]:
label_count = {}
for y in y_test:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [17]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_test, y_test)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.4386704119850187
Dev/test accuracy: 0.42779291553133514


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [18]:
import random
def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        y_preds.append(run_classifier(x))
        if y != y_preds[-1]:
            error_ids.append(i)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]
        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}\n')

In [19]:
find_errors(x_train, y_train)

-LRB- Stevens is -RRB- so stoked to make an important film about human infidelity and happenstance that he tosses a kitchen sink onto a story already overladen with plot conceits .
true label: -1
predicted label: 1

True Hollywood Story .
true label: 0
predicted label: 1

-LRB- Toback 's -RRB- fondness for fancy split-screen , stuttering editing and pompous references to Wittgenstein and Kirkegaard ... blends uneasily with the titillating material .
true label: -1
predicted label: 1

A `` Home Alone '' film that is staged like `` Rosemary 's Baby , '' but is not as well-conceived as either of those films .
true label: -1
predicted label: 1

Behind the glitz , Hollywood is sordid and disgusting .
true label: -1
predicted label: 1

